## 한국수출입은행 - 원화 환율(최근까지) 및 DB 저장
+ 작성: 임경호
+ 한국수출입은행 Open API https://www.koreaexim.go.kr/ir/HPHKIR019M01

### 1. 데이터를 가져올 날짜 설정
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [1]:
import requests
import pandas as pd

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date FROM fx_eximbank ORDER BY date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

date_from = myutil.get_next_day(last_date)
date_to = myutil.get_previous_day()

In [2]:
print(date_from, date_to)

20230902 20230907


### 2. API를 활용한 데이터 수집 (한국수출입은행 Open API)

In [3]:
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        # print(df_fx['result'].unique())
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print(f'{date_str}: Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

In [4]:
df_fx_all.shape

(92, 11)

In [5]:
df_fx_all

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm
0,20230904,AED,355.55,362.74,359.15,359,0,0,359,359.15,아랍에미리트 디르함
1,20230904,AUD,842.5,859.53,851.02,851,0,0,851,851.02,호주 달러
2,20230904,BHD,"3,464.3","3,534.29","3,499.3","3,499",0,0,"3,499","3,499.3",바레인 디나르
3,20230904,BND,964.3,983.79,974.05,974,0,0,974,974.05,브루나이 달러
4,20230904,CAD,960.54,979.95,970.25,970,0,0,970,970.25,캐나다 달러
...,...,...,...,...,...,...,...,...,...,...,...
87,20230907,SAR,351.71,358.82,355.27,355,0,0,355,355.27,사우디 리얄
88,20230907,SEK,118.69,121.08,119.89,119,0,0,119,119.89,스웨덴 크로나
89,20230907,SGD,967.31,986.86,977.09,977,0,0,977,977.09,싱가포르 달러
90,20230907,THB,37.11,37.86,37.49,37,0,0,37,37.49,태국 바트


### 3. 데이터 전처리

In [6]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

In [ ]:
# 특정 통화 환율 확인
# df_fx_all_save[df_fx_all_save['cur_unit'].str.contains('USD')]

In [7]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [8]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230904,AED,355.55,362.74,359.15,359,359,359.15
1,20230904,AUD,842.5,859.53,851.02,851,851,851.02
2,20230904,BHD,3464.3,3534.29,3499.3,3499,3499,3499.3
3,20230904,BND,964.3,983.79,974.05,974,974,974.05
4,20230904,CAD,960.54,979.95,970.25,970,970,970.25
...,...,...,...,...,...,...,...,...
87,20230907,SAR,351.71,358.82,355.27,355,355,355.27
88,20230907,SEK,118.69,121.08,119.89,119,119,119.89
89,20230907,SGD,967.31,986.86,977.09,977,977,977.09
90,20230907,THB,37.11,37.86,37.49,37,37,37.49


In [ ]:
# 특정 통화 환율 확인
# df_fx_all_save[df_fx_all_save['cur_code'].str.contains('USD')]

### 4. 테이블에 데이터 저장 (mdw.weather)

In [9]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()